### Testeo de Chatbot

Primero se debe importar tanto el Expert Agent como el RAGEmbeddingEngine ya que el primero recibe como parametro una rag_tool

In [1]:
from src.expert_agent import ExpertAgent
from src.rag_llamaindex import RAGEmbeddingEngine

d:\Users\Alejandro\Documents\Python Scripts\ChallangeDs\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import logging
logging.basicConfig(level=logging.INFO)

La tool recibe como parametro default el path donde esta el index_store de llama-index para especificar la ruta del directorio donde se almacenará o desde donde se cargará el índice de embeddings de un texto procesado

In [3]:
rag_tool = RAGEmbeddingEngine()

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-large
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
INFO:llama_index.core.indices.loading:Loading all indices.
INFO:root:Índice cargado exitosamente.


Una vez inicializado el rag_tool, lo podemos pasar instanciando un ExpertAgent. Esta division obedece a que la inicializacion del rag_tool tiene cierta duracion y dado que el agente se deberia poder reinstanciar agilmente se prefiere que esta herramienta se instancie de forma separada

In [4]:
expert = ExpertAgent(rag_tool=rag_tool)

INFO:root:Expert Agent initialized


Si se desea que el agente pierda el chat history y por tanto la memoria de la conversacion, se debe volver a instanciar

In [5]:
response = expert.chat("Como se llama el lobo de Sansa?")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:root:Assistant requested tool calls: [ToolCall(function=Function(name='consult_expert', arguments={'query': 'Nombre del lobo de Sansa en Juego de Tronos'}))]
INFO:root:Running tool consult_expert with args {'query': 'Nombre del lobo de Sansa en Juego de Tronos'}
INFO:root:Running 'consult_expert' with query: Nombre del lobo de Sansa en Juego de Tronos
INFO:root:Running 'search' with query: Nombre del lobo de Sansa en Juego de Tronos
Batches: 100%|██████████| 1/1 [00:03<00:00,  3.53s/it]
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


In [6]:
print(response)

El nombre del lobo de Sansa es Dama. (REFERENCIAS PAGINA 59, 104)


In [7]:
for message in expert.chat_history:
    print(message)
    print("-----")

role='system' content="Eres un asistente conversacional. Tu única función es ser una interfaz amigable entre el usuario y la tool 'consult_expert'.\n\nTu rol es exclusivamente hacer esta tarea respecto a la tematica de Juego de Tronos (primer libro). No respondes preguntas sobre otros temas.\n\nPara cada pregunta del usuario relacionada a juego de tronos debes: \n1. Consultar sobre preguntas de la tematica de Juego de Tronos a la tool 'consult_expert'\n2. La tool 'consult_expert' devolvera una respuesta directa a la pregunta formulada y los numeros de pagina de donde se puede encontrar la información. \n4. Dar una respuesta amigable que transmita la informacion proporcionada por la tool, sin agregar información adicional. \n\nReglas que debes seguir: \n- NUNCA respondas sobre el libro sin consultar la tool 'consult_expert'\n- SIEMPRE genera tus respuestas sobre Juego de Tronos solo con la información proporcionada por la tool 'consult_expert' e indicas las paginas de donde proviene la 

En primer lugar se intento darle al agente no solo la respuesta generada por el RAG sino tambien los fragmentos de las paginas recuparadas pero este enfoque genero que la atencion del agente se viese menoscabada y comenzo a hallucinar

Respuestas incluyendo los fragmentos de texto devueltos por el RAG como respuesta del tool call:

- A quien le vertieron oro fundido en la cabeza? response = Viserys Targaryen le vertieron oro fundido en la cabeza después de que Dany lo atavió con un caldero lleno de oro como una corona. Este evento ocurrió cuando Viserys Targaryen se sintió amenazado por su hermana Dany y pretendía forzarle a ella a casarse contigo.

- Y que dijo Dany cuando eso ocurrio? response = Dany dijo "¡Corona!" cuando Viserys Targaryen se sentía amenazado por ella y pretendía forzarle a casarse con él. Ella respondió de manera tranquila y contempló su propia imagen en un espejo azogado después de que su hermano la vestiera para presentarla ante Khal Drogo, quien era conocido por ser rico y tener un collar de oro con incrustaciones de amatistas en las muñecas.

- Describeme a Hielo, la espada de la familia stark response = Hielo es una espada valyriana de acero oscuro y longitud considerable, similar a las de los grandes guerreros de la antigüedad. Su filo es tan afilado que parece tener un brillo tembloroso, y está forjada con encantamientos que le dan un equilibrio excepcional. La espada tiene un peso especial en la mano del dueño, lo que la hace más ligera de lo que sería si se esgrimiera con una o con dos manos. La espada tiene un brillo especial y un equilibrio perfecto, lo que permite al dueño esgrimirla con facilidad. También tiene un significado especial para Jon, ya que su padre, Lord Eddard, le dio la espada cuando era joven y la llamó "espada de bastardos" debido a su relación con su madre. En resumen, Hielo es una espada valyriana única y especial que ha sido heredada por varias generaciones de la Casa Mormont. Su filo es afilado y está forjado con encantamientos que le dan un equilibrio excepcional.

- Quien es la hermana fallecida de Ned Stark? response = Basándome en la información proporcionada, parece que Lyanna es la hermana menor de Eddard Stark y Brandon, y su fallecimiento se menciona en el capítulo 552 de la novela "Casa Stark" de George R.R. Martin. La hermana de Lyanna era Arya, que tenía nueve años en ese momento.

- Quien es el mejor amigo de Jon Snow? response = Basándome en la información proporcionada por la tool 'consult_expert', parece que el mejor amigo de Jon Snow es Samwell Tarly. Sam es descrito como un hombre gordo, torpe y asustadizo, pero también con cerebro, y se le considera hermano adoptivo de Jon después de que su padre, Ned Stark, murió. Jon y Sam comparten una fuerte amistad y confianza mutua, y son mencionados en varias ocasiones en el libro como estar juntos en diferentes situaciones.

8 preguntas:

- A quien le vertieron oro fundido en la cabeza?

- Que dice Dany cuando eso ocurre?

- Describeme a Hielo, la espada de la familia Stark

- Como se llamaba la hermana fallecida de Eddard Stark?

- Quien es el mejor amigo de Jon Snow?

- Como se llamaba el instructor de Arya Stark?

- Como esta compuesta la guardia real?

- ¿Como se llama el lobo de Jon Snow?

- Como se llama el lobo de Sansa?

- Que tenia que hacer Daenerys para que su hijo nazca sano segun la tradicion dothraki?

Respuestas SIN FRAGMENTOS de texto. Solo la respuesta generada y paginas de referencia:

- La persona a quien se le vertieron oro fundido en la cabeza es Viserys Targaryen. (REFERENCIAS PAGINA 343, 361)

- Dany se sentía extrañamente tranquila cuando su hermano Viserys se cubrió la cara con un yelmo de hierro cubierto de oro fundido, y el sonido que emitió fue más ritual que grito humano. (REFERENCIAS PAGINA 343, 28)

- La espada de Ned Stark se llamaba "Hielo" (también conocida como "Ice"). Fue la espada que llevó su mano, el hombre que la utilizaba había sido ejecutado por orden de Ned Stark y su nombre era Grey Worm.

- La hermana fallecida de Ned Stark fue Lyanna. (REFERENCIAS PAGINA 141, 552)

- La mejor amistad de Jon Snow es con Samwell Tarly. La relación entre ellos es muy cercana y Sam muestra un gran apoyo y lealtad hacia Jon, incluso en momentos difíciles. (REFERENCIAS PAGINA 41, 188)

- El instructor de Arya Stark se llamaba Syrio Forel. Puedes encontrar esta información en las páginas 220 y 240 del libro.

- La Guardia Real del rey está compuesta por varios caballeros que han jurado fidelidad al trono. Entre ellos se encuentran Ser Barristan Selmy, Lord Comandante; Ser Jaime Lannister, también conocido como el Matarreyes; y otros miembros destacados como Lancel y Tyrek Lannister. Estos caballeros protegen al monarca y han jurado lealtad al rey. (REFERENCIAS PÁGINA 4, 551)

- El lobo de Jon Snow se llama Fantasma. Puedes encontrar esta información en las páginas 59 y 87 del libro.

- El lobo de Sansa se llama Dama. (REFERENCIAS PAGINA 59, 104)

- Para que su hijo naza sano según la tradición Dothraki, Daenerys debe consumir el corazón del semental salvaje entero sin atragantarse ni vomitar. Esto es para evitar presagios no favorables. (Referencias página 336 y 482)
